# Problem Set Week 1: Yield-to-Maturity Calculation

This notebook contains solutions to Problem Set Week 1 on calculating yield-to-maturity (YTM) for a fixed rate bullet bond using various optimization methods.


In [1]:
# Import necessary packages
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import fixed_income_derivatives_E2025 as fid

## Bond Setup

We consider a fixed rate bullet bond with:
- Annual coupon rate: 6% (paid semi-annually, so 3% per period)
- Principal: 100 USD
- Maturity: 10 years
- Current market price: 98.74 USD


We will try to minimize the squared error brtween the true present value (The current market price) and the present value of the future cash-flows, as a function of YTM. We consider the following minimization problem:

$$
\min{y} SE=(\prod - \hat{\prod}(y\mid C))^2 
$$


## **Problem a) Write a function, with inputs of ytm, vector of times of CFs, and vector of CFs. Should return the PV of the CFs**

We want to write up a function, which finds the discrete-compounding, of the discounted cashflows, i.e.

$$
\hat{\prod}_y(C) = \sum_{i} \frac{C_i}{(1+y)^T_i}

$$

In [27]:
def present_value(y, T, C):
    """
    Calculate present value of cash flows using discrete compounding.
    
    Parameters:
    y : float - yield-to-maturity
    T : array - times of cash flows
    C : array - cash flows
    
    Returns:
    float - present value
    """
    # Ensure y is not too negative (1+y must be positive)
    if y <= -1:
        return np.inf  # Return infinity for invalid yields
    return np.sum(C / (1 + y)**T)

# Test the function
test_y = 0.05
test_T = np.array([0.5, 1.0, 1.5, 2.0])
test_C = np.array([3.0, 3.0, 3.0, 103.0])
test_PV = present_value(test_y, test_T, test_C)
print(f"Present value of test bond: {test_PV:.2f}")


Present value of test bond: 102.00


## **Problem b) Function to find squared deviation frmo true PV and PV of specific YTM**

We need to implement:
$$
\min{y} SE=(\prod - \hat{\prod}(y\mid C))^2 
$$

In [28]:
def squared_error_y(y, Pi, T, C):
    y = float(np.atleast_1d(y)[0])
    pv_hat = present_value(y, T, C)
    return (Pi - pv_hat)**2

# test the function
test_y = 0.05
test_T = np.array([0.5, 1.0, 1.5, 2.0])
test_C = np.array([3.0, 3.0, 3.0, 103.0])
test_Pi = 98.74
test_SE = squared_error_y(test_y, test_Pi, test_T, test_C)
print(f"Squared error: {test_SE}")

Squared error: 10.609125660440753


## **Problem c) Compute the YTM by minimising the squared error using Nelder-Mead and Powell**

In [30]:
# 1) Define the bullet bond

# Bond parameters
R = 0.06  # Annual coupon rate (6%)
K = 100      # Principal amount
T_N = 10       # Years to maturity
alpha = 0.5  # Semi-annual compounding
Pi = 98.74  # Current market price

# Number of cash flows: N = T_N/alpha + 1 = 10/0.5 + 1 = 21
N = int(T_N/alpha + 1)

# Define cash flow periods: T = [0, 0.5, 1.0, 1.5, ..., 10.0]
T = np.array([i*alpha for i in range(0, N)])

# Define cash flows: coupon payments at each period, plus principal at maturity
C = np.zeros(N)
for i in range(0, N):
    C[i] += R*alpha*K  # Coupon payment at each period
C[-1] += K  # Add principal repayment at final period


y_init = R
args = (Pi, T, C)

# --- Nelder–Mead ---
result_nelder = minimize(
    squared_error_y,
    y_init,
    args=args,
    method="nelder-mead",
    options={"xatol": 1e-8, "disp": False}
)

print(f"Nelder–Mead YTM: {result_nelder.x[0]:.6f}\n")

# --- Powell ---
result_powell = minimize(
    squared_error_y,
    y_init,
    args=args,
    method="powell",
    bounds=[(0.0, None)],          # keeps (1+y) > 0
    options={"xtol": 1e-8, "disp": False}
)

print(f"Powell YTM: {result_powell.x[0]:.6f}")




Nelder–Mead YTM: 0.066969

Powell YTM: 0.066969


## **d) Finding partial derivatives of SE wrt y and y^2**


We define:
$$
\begin{aligned}SE= & \left(\Pi-\hat{\Pi}\left(y\mid C\right)\right)^{2}\\
= & \left(\Pi-\sum_{i}\frac{C_{i}}{(1+y)^{T_{i}}}\right)^{2}\\
= & \left(\Pi-\sum_{i}C_{i}(1+y)^{-T_{i}}\right)^{2}
\end{aligned}
$$

Then taking the derivative wrt y yields:
$$
\begin{aligned}\frac{\partial SE}{\partial y}= & \;2\left(\Pi-\sum_{i}C_{i}(1+y)^{-T_{i}}\right)\left(\sum_{i}\frac{T_{i}C_{i}}{(1+y)^{T_{i}+1}}\right)\\
= & \;2\left(\Pi-\hat{\Pi}\left(y\mid C\right)\right)\left(\sum_{i}\frac{T_{i}C_{i}}{\left(1+y\right)^{T_{i}+1}}\right).
\end{aligned}

$$

Finding the second order derivative:
$$
\begin{aligned}\frac{\partial^{2}SE}{\partial y\partial y}= & 2\left(\sum_{i}\frac{C_{i}T_{i}}{\left(1+y\right)^{T_{i}+1}}\right)^{2}-2\left(\Pi-\hat{\Pi}\left(y\mid C\right)\right)\left(\sum_{i}\frac{C_{i}T_{i}\left(T_{i}+1\right)}{\left(1+y\right)^{T_{i}+2}}\right)\end{aligned}
$$

## **e)Write a function in Python that returns the first order derivative of the objective function and find the yield-to-maturity using the ’BFGS’ method and the first order derivative.**

In [ ]:
# Defininig the partial derivatives
def partial_derivative_y(y, Pi, T, C):
    y = float(np.atleast_1d(y)[0])
    pv_hat = present_value(y, T, C)
    dpv_dy = np.sum(-C * T / (1 + y)**(T + 1))          # PV'(y)
    return -2 * (Pi - pv_hat) * dpv_dy                 # SE'(y)


In [35]:
# Problem 1e - Finding the yield to maturity using the first order derivative and 'BFGS'

y_init = R  # Initial guess for yield
args = (Pi, T, C)

result = minimize(
    fun=lambda y, Pi, T, C: (Pi - present_value(y, T, C))**2, 
    x0=y_init,
    args=args,
    method='BFGS', 
    jac=partial_derivative_y
)
y = result.x[0]

print(f"BFGS with 1. order derivative. ytm: {y}")



BFGS with 1. order derivative. ytm: 0.06696883208690854


## **Write a function in Python that returns the second order derivative of the objective function and find the yield-to-maturity using the ’Newton-CG’ method and both the first- and second order derivative.**



In [36]:

def second_order_derivative_y(y, Pi, T, C):
    y = float(np.atleast_1d(y)[0])
    pv_hat = present_value(y, T, C)
    dpv_dy  = np.sum(-C * T / (1 + y)**(T + 1))         # PV'(y)
    ddpv_ddy = np.sum(C * T * (T + 1) / (1 + y)**(T + 2)) # PV''(y)
    return 2 * (pv_hat - Pi) * ddpv_ddy + 2 * (dpv_dy**2) # SE''(y)

In [37]:
y_init = R
args = (Pi, T, C)

result = minimize(
    fun=lambda y, Pi, T, C: (Pi - present_value(y, T, C))**2, 
    x0=y_init,
    args=args,
    method='Newton-CG',
    jac=partial_derivative_y,
    hess=second_order_derivative_y
)
y = result.x[0]

print(f"Newton-CG with 1. and 2. order derivatives. ytm: {y}")

Newton-CG with 1. and 2. order derivatives. ytm: 0.06696883165863443


## **g) Use the ’trust-constr’ method to find the yield-to-maturity when imposing that y ∈ [0, 0.12] and y ∈ [0.08, 0.12].**

In [47]:
import numpy as np
from scipy.optimize import minimize, Bounds

y_init = R
args = (Pi, T, C)

lb, ub = 0.01, 0.30
bounds = Bounds([lb], [ub])          # trust-constr expects vector bounds

result = minimize(
    fun=lambda y, Pi, T, C: (Pi - present_value(float(np.atleast_1d(y)[0]), T, C))**2,
    x0=y_init,
    args=args,
    method="trust-constr",
    jac=lambda y, Pi, T, C: np.array([partial_derivative_y(y, Pi, T, C)]),          # shape (1,)
    hess=lambda y, Pi, T, C: np.array([[second_order_derivative_y(y, Pi, T, C)]]),  # shape (1,1)
    bounds=bounds
)

y = result.x[0]
print(f"Trust-constr with bounds [{lb}, {ub}]. ytm: {y:.6f}")


Trust-constr with bounds [0.01, 0.3]. ytm: 0.066969
